In [21]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics import mean_squared_error, r2_score  

In [22]:
data = pd.read_excel('ЛР1. Исходные данные.xlsx')
n = len(data.index) # n - кол-во объектов
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
data.head()

,Y,х1,х2,х3,х4,х5,х6,x7,x8,x9
0,59.9,7.0,15.9,8.2,5.1,13.8,229,172,20.0,2104
1,55.5,7.2,18.2,7.4,6.1,14.3,146,167,29.1,2489
2,55.3,7.9,19.7,6.4,4.7,19.8,174,144,22.8,2428
3,55.8,7.7,20.8,6.9,5.2,17.1,128,111,42.7,2494
4,60.1,9.2,15.9,7.8,5.3,16.7,169,148,22.7,2094


In [23]:
# Нормирование данных (Z-нормализация)

mean = np.mean(X, axis=0)
stddev = np.std(X, axis=0)
norm_X = (X - mean) / stddev

x0 = np.ones((n,1)) # доп. единчиный вектор-столбец
m = len(data.columns) # m - кол-во признаков
norm_X.insert(0, "x0", x0)

norm_X.head()

,x0,х1,х2,х3,х4,х5,х6,x7,x8,x9
0,1.0,-1.025937,0.495203,1.763429,0.451789,-1.299494,1.121916,0.376805,-1.040047,0.288039
1,1.0,-0.954255,1.323542,0.415279,1.318235,-1.167518,-0.274620,0.249229,-0.276237,0.867287
2,1.0,-0.703371,1.863763,-1.269910,0.105211,0.284220,0.196501,-0.337621,-0.805029,0.775510
3,1.0,-0.775053,2.259925,-0.427316,0.538434,-0.428451,-0.577483,-1.179623,0.865282,0.874809
4,1.0,-0.237444,0.495203,1.089354,0.625079,-0.534032,0.112372,-0.235560,-0.813422,0.272993


In [24]:
# Аналитический метод расчета весов линейной регрессии

Ω = np.linalg.inv(norm_X.T.dot(norm_X)).dot(norm_X.T).dot(Y) 
print('Веса: ', Ω)

R2_1 = r2_score(Y, norm_X.dot(Ω))
MSE1 = mean_squared_error(Y, norm_X.dot(Ω))
print('R2 score = ', R2_1)
print('MSE = ', MSE1)

Веса:  [58.36428571 -2.53679535 -2.10471414  0.9343742  -2.09746843 -0.2268894
 -0.35014402 -1.07500846 -0.50433838 -0.65675583]
R2 score =  0.5987624854317046
MSE =  3.5838289145618982


In [7]:
# Корреляционная матрица

corr = norm_X.corr()
corr

,x0,х1,х2,х3,х4,х5,х6,x7,x8,x9
x0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
х1,NaN,1.000000,-0.690889,-0.153449,-0.617002,0.230033,-0.378251,-0.350215,0.617462,-0.205805
х2,NaN,-0.690889,1.000000,-0.042322,0.157473,-0.205796,0.274620,0.141806,-0.351087,0.225195
х3,NaN,-0.153449,-0.042322,1.000000,0.362014,-0.160446,0.294956,0.146964,-0.205459,-0.298880
х4,NaN,-0.617002,0.157473,0.362014,1.000000,0.106949,0.163901,0.195091,-0.491273,0.291752
х5,NaN,0.230033,-0.205796,-0.160446,0.106949,1.000000,-0.287172,-0.205484,0.190996,0.154447
х6,NaN,-0.378251,0.274620,0.294956,0.163901,-0.287172,1.000000,0.566285,-0.553216,-0.088237
x7,NaN,-0.350215,0.141806,0.146964,0.195091,-0.205484,0.566285,1.000000,-0.527482,0.190287
x8,NaN,0.617462,-0.351087,-0.205459,-0.491273,0.190996,-0.553216,-0.527482,1.000000,0.017279
x9,NaN,-0.205805,0.225195,-0.298880,0.291752,0.154447,-0.088237,0.190287,0.017279,1.000000


In [8]:
# Число обусловленности

cond = np.linalg.cond(norm_X)
cond

5.830522770123726

In [9]:
# Нахождение регрессионных остатков
E = Y - norm_X.dot(Ω)

# Проверка нормальности распределения остатков
print(sp.stats.normaltest(E))

# Проверка гипотезы о равенстве нулю среднего значения остатков
print(sp.stats.wilcoxon(E))

print(E.head())

NormaltestResult(statistic=51.15854118721361, pvalue=7.781510899234208e-12)
WilcoxonResult(statistic=744.0, pvalue=0.6595876411823418)
0   -0.557010
1    0.214719
2    0.355000
3    1.198095
4    1.902847
dtype: float64


In [25]:
# Метод градиентного спуска

# Инициализация весов
W_new = np.ones(m)
Y_predict = norm_X.dot(W_new)
learning_rate = 0.1
iter_num = 100
arr_S =[]
for _ in range(iter_num):
    
    # Вычисление предсказанного y
    Y_predict = norm_X.dot(W_new)
    E = Y - Y_predict
    
    # Ф-ия потерь MSE   
    S = np.mean(E ** 2)
    arr_S.append(S)
    
    # Градиенит ф-ии потерь MSE   
    dS_dW = 2/n * E.dot(-norm_X)    
    W_new -= learning_rate * dS_dW
R2_2 = r2_score(Y, Y_predict)
MSE2 = mean_squared_error(Y, Y_predict) 
print('Веса: ', W_new)
print('\nМассив ошибок: ', arr_S)
print('\nR2 score = ', R2_2)
print('\nMSE: ', MSE2)


Веса:  x0    58.364286
х1    -2.070876
х2    -1.787720
х3     0.799946
х4    -1.700127
х5    -0.304601
х6    -0.351257
x7    -0.939535
x8    -0.416351
x9    -0.819362
dtype: float64

Массив ошибок:  [3313.0664510154756, 2121.02051195486, 1359.6199469713133, 872.5289362552066, 560.7796255076752, 361.2011430758347, 233.40188494368448, 151.5429754905079, 99.09144457360959, 65.4671106122202, 43.89839575214887, 30.050814092355218, 21.149531680092302, 15.41793686868812, 11.718370302974275, 9.322199632174634, 7.762659959194364, 6.740659524082902, 6.064482597584589, 5.611187727207158, 5.301892660525347, 5.085948413556275, 4.9308038950255035, 4.815513767246274, 4.726579040167026, 4.655281481515964, 4.595974770964555, 4.544988535694605, 4.499925084067636, 4.459207830160894, 4.421791092178105, 4.386973403788474, 4.354277260856967, 4.323371536691549, 4.294021325200448, 4.266055434124724, 4.239345251455041, 4.21379095256126, 4.189312455041338, 4.165843452001466, 4.143327447600613, 4.1217150998491, 

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(norm_X, Y)
y_pred = model.predict(norm_X)
y_pred, (y_pred - Y).head(20)

(array([60.45701043, 55.2852806 , 54.94500023, 54.60190537, 58.19715269,
        58.89751528, 56.53587628, 57.86213988, 57.36440261, 58.26029103,
        59.64422627, 59.91985413, 59.07954187, 58.29367035, 55.83753442,
        56.66828733, 57.58300805, 61.03725825, 58.82991279, 59.46534797,
        59.05543165, 62.67691793, 58.76886833, 60.68653282, 60.27120278,
        60.99948638, 61.89666978, 62.64270972, 63.05398298, 61.41154129,
        59.64227192, 60.26530523, 59.86865841, 60.32643687, 61.50947957,
        59.07803036, 60.55449444, 56.36618325, 57.13383751, 57.99074979,
        58.42333636, 52.1270278 , 57.41804939, 53.97430284, 56.62825112,
        57.38315335, 57.38145209, 56.65560701, 58.14948504, 57.22068403,
        55.94844866, 58.03161119, 57.12961995, 56.43221246, 53.61379178,
        58.91896014]),
 0     0.557010
 1    -0.214719
 2    -0.355000
 3    -1.198095
 4    -1.902847
 5     0.397515
 6    -0.864124
 7    -0.637860
 8    -0.935597
 9     0.060291
 10    3.14422

In [12]:
import statsmodels.api as sm
statmodel = sm.OLS(Y, norm_X).fit()
statmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     7.627
Date:                Fri, 06 Oct 2023   Prob (F-statistic):           9.62e-07
Time:                        15:39:07   Log-Likelihood:                -115.20
No. Observations:                  56   AIC:                             250.4
Df Residuals:                      46   BIC:                             270.7
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x0            58.3643      0.279    209.099      0.000      57.802      58.926
х1            -2.5368      0.637     -3.982      0.000      -3.819      -1.254
х2            -2.1047      0.478     -4.399      0.000      -3.068      -1.142
х3             0.9344      0.368      2.539      0.015       0.194       1.675
х4            -2.0975      0.548     -3.827      0.000      -3.201      -0.994
х5            -0.2269      0.320     -0.710      0.482      -0.871       0.417
х6            -0.3501      0.390     -0.897      0.374      -1.136       0.436
x7            -1.0750      0.400     -2.686      0.010      -1.881      -0.269
x8            -0.5043      0.451     -1.119      0.269      -1.412       0.403
x9            -0.6568      0.383     -1.715      0.093      -1.427       0.114
==============================================================================
Omnibus:                       51.159   Durbin-Watson:                   2.496
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              336.016
Skew:                           2.285   Prob(JB):                     1.08e-73
Kurtosis:                      14.096   Cond. No.                         5.83
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""